In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('gurgaon_properties_cleaned_v1.csv')

In [5]:
df.isnull().sum()

property_type         0
society               1
sector                0
price                18
price_per_sqft       18
area                 18
areaWithType          0
bedRoom               0
bathroom              0
balcony               0
additionalRoom        0
floorNum             19
facing             1105
agePossession         1
nearbyLocations     177
furnishDetails      981
features            635
dtype: int64

In [6]:
df.duplicated().sum()

122

In [7]:
df.head(1)
# focus is on -> areaWithType, additionalRoom, agePossession, furnishDetails, features 



,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features
0,flat,ramprasta awho,sector 95,0.92,5257.0,1750.0,Built Up area: 1750 (162.58 sq.m.)Carpet area:...,3,3,3+,not available,13.0,North-East,undefined,NaN,NaN,NaN


In [13]:
#areaWithType
import re

# This function extracts the Super Built up area
def get_super_built_up_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [14]:
# This function extracts the Built Up area or Carpet area
def get_area(text, area_type):
    match = re.search(area_type + r'\s*:\s*(\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [15]:


# This function checks if the area is provided in sq.m. and converts it to sqft if needed
def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{} \((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(1))
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value



In [16]:


# Extract Super Built up area and convert to sqft if needed
df['super_built_up_area'] = df['areaWithType'].apply(get_super_built_up_area)
df['super_built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['super_built_up_area']), axis=1)

# Extract Built Up area and convert to sqft if needed
df['built_up_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Built Up area'))
df['built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['built_up_area']), axis=1)

# Extract Carpet area and convert to sqft if needed
df['carpet_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Carpet area'))
df['carpet_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['carpet_area']), axis=1)



In [18]:
df[['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].sample(5)

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
3114,2.17,flat,1550.0,Carpet area: 1549 (143.91 sq.m.),NaN,NaN,1549.0
3791,1.60,flat,2545.0,Super Built up area 2545(236.44 sq.m.),2545.0,NaN,NaN
3393,1.25,flat,1744.0,Super Built up area 1744(162.02 sq.m.),1744.0,NaN,NaN
759,1.60,flat,1950.0,Super Built up area 1950(181.16 sq.m.)Carpet a...,1950.0,NaN,1161.0
1222,0.95,flat,1385.0,Super Built up area 1385(128.67 sq.m.)Built Up...,1385.0,940.0,845.0


In [19]:
df[~((df['super_built_up_area'].isnull()) | (df['built_up_area'].isnull()) | (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].shape

(534, 7)

In [20]:
df[df['areaWithType'].str.contains('Plot')][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].head(5)

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2,0.42,house,50.0,Plot area 50(4.65 sq.m.),NaN,NaN,NaN
3,6.50,house,1836.0,Plot area 215(179.77 sq.m.)Built Up area: 215 ...,NaN,215.0,NaN
15,8.50,house,4000.0,Plot area 4000(371.61 sq.m.),NaN,NaN,NaN
16,3.51,house,2844.0,Plot area 316(264.22 sq.m.),NaN,NaN,NaN
18,2.40,house,2000.0,Plot area 2000(185.81 sq.m.),NaN,NaN,NaN
